In [6]:
import os
import pickle
import pandas as pd

In [7]:
# check the extracted skelton data
with open('/skeleton_all/B/B001_GMS_1_1_3.pkl', 'rb') as pickle_file:
    sample = pickle.load(pickle_file)
print(sample)

{'keypoint': array([[[[ 595.08264,  540.9249 ],
         [ 599.45215,  527.8165 ],
         [ 590.71326,  527.8165 ],
         ...,
         [ 573.23535,  807.4629 ],
         [ 503.3237 ,  877.3744 ],
         [ 551.38794,  890.4829 ]],

        [[ 598.9317 ,  540.81335],
         [ 607.70074,  527.6598 ],
         [ 594.5472 ,  527.6598 ],
         ...,
         [ 572.6245 ,  812.6541 ],
         [ 502.47205,  869.653  ],
         [ 555.08636,  891.5756 ]],

        [[ 598.8917 ,  539.9436 ],
         [ 598.8917 ,  531.128  ],
         [ 598.8917 ,  531.128  ],
         ...,
         [ 576.85266,  808.8209 ],
         [ 510.7353 ,  866.12256],
         [ 554.81354,  888.16174]],

        ...,

        [[1027.0935 ,  279.80475],
         [1035.6987 ,  266.8968 ],
         [1014.18555,  262.59412],
         ...,
         [ 979.7643 ,  542.2664 ],
         [1001.2776 ,  615.4115 ],
         [ 971.15906,  624.0168 ]],

        [[1029.0168 ,  280.13272],
         [1037.6172 ,  267.2322 ],

In [10]:
# load the ground truth label file for each skeleton action video
ev = pd.read_csv('../gms_eval_v3.csv')
ev = ev.drop(columns=['Type', '1차 작업자', '행동번호'])
ev = ev[['ID', 'action', '1차 평가']]
ev = ev.dropna(axis=0)
ev = ev[ev.ID.str[0] == 'B']

In [ ]:
# add label to each data and save 
errCnt = 0
errLi = []
totalerr = []
for root, dirs, files in os.walk('skeleton_all/B/'):
    for file in files:
        with open(f'{root}/{file}', 'rb') as pickle_file:
            content = pickle.load(pickle_file)
        action = file[5:10]+'_'+file[13:14]
        actionNum = file[9:10]
        camera = file[11:12]
        name = file[:14]
        pid = file[:4]
        if (camera == '1') & (actionNum == '4'):
            try:
                label = max(1, int(ev[(ev['action'] == action) & (ev['ID'] == pid)]['1차 평가']))
                label = label - 1
                content['label'] = label
            except Exception as e:
                print(e)
                errCnt += 1
                errLi.append(name)
                totalerr.append(name)
                continue
            with open(f'my_skeleton_labeled/B_4_1/{name}.pkl', 'wb') as fw:
                pickle.dump(content, fw)